In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
# import numpy as np

In [3]:
database_type = "/training"

#### Reading in Training Tables


In [4]:
lab = pd.read_csv('../eICU' + database_type + '/lab.csv')
lab

,patientunitstayid,labresultoffset,creatinine,wbcx1000,lactate
0,141168,1701,173.12,NaN,NaN
1,141168,2026,2.95,19.8,12.2
2,141168,1133,2.30,14.7,NaN
3,141168,516,1.95,9.8,NaN
4,141178,-280,0.70,7.6,NaN
...,...,...,...,...,...
1292236,3353254,5558,1.67,11.7,NaN
1292237,3353254,4144,1.61,NaN,NaN
1292238,3353254,-256,2.38,13.5,NaN
1292239,3353263,-7,1.06,6.5,NaN


In [5]:
vitalperiodic = pd.read_csv('../eICU' + database_type + '/vitalPeriodic.csv')
vitalperiodic

,patientunitstayid,observationoffset,temperature,heartrate,respiration,systemicsystolic
0,141168,2059,NaN,92.0,NaN,NaN
1,141168,1289,NaN,118.0,NaN,NaN
2,141168,1794,NaN,78.0,NaN,NaN
3,141168,1374,NaN,118.0,NaN,NaN
4,141168,2039,NaN,92.0,NaN,NaN
...,...,...,...,...,...,...
146671637,3353263,504,NaN,87.0,36.0,NaN
146671638,3353263,534,NaN,81.0,21.0,NaN
146671639,3353263,549,NaN,78.0,14.0,NaN
146671640,3353263,544,NaN,85.0,25.0,NaN


In [6]:
# The original dataframes were deleted to free up resources.  
merge = pd.merge(vitalperiodic, lab,  how='outer', left_on=['patientunitstayid','observationoffset'], right_on = ['patientunitstayid','labresultoffset'])
del lab, vitalperiodic
merge 

,patientunitstayid,observationoffset,temperature,heartrate,respiration,systemicsystolic,labresultoffset,creatinine,wbcx1000,lactate
0,141168,2059.0,NaN,92.0,NaN,NaN,NaN,NaN,NaN,NaN
1,141168,1289.0,NaN,118.0,NaN,NaN,NaN,NaN,NaN,NaN
2,141168,1794.0,NaN,78.0,NaN,NaN,NaN,NaN,NaN,NaN
3,141168,1374.0,NaN,118.0,NaN,NaN,NaN,NaN,NaN,NaN
4,141168,2039.0,NaN,92.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
147832933,3353254,NaN,NaN,NaN,NaN,NaN,5558.0,1.67,11.7,NaN
147832934,3353254,NaN,NaN,NaN,NaN,NaN,4144.0,1.61,NaN,NaN
147832935,3353254,NaN,NaN,NaN,NaN,NaN,-256.0,2.38,13.5,NaN
147832936,3353263,NaN,NaN,NaN,NaN,NaN,-7.0,1.06,6.5,NaN


In [7]:
intakeOutputUrine = pd.read_csv('../eICU' + database_type + '/intakeOutputUrine.csv')
intakeOutputUrine

,patientunitstayid,intakeoutputoffset,urineoutputbyweight
0,141179,1420,3.968254
1,141179,933,2.834467
2,141179,1703,5.668934
3,141179,678,3.968254
4,141194,12201,1.353180
...,...,...,...
3331676,3353254,4169,19.070322
3331677,3353254,5326,4.767580
3331678,3353254,2626,10.131108
3331679,3353254,3101,15.494636


In [ ]:
merge1 = pd.merge(merge, intakeOutputUrine,  how='outer', left_on=['patientunitstayid','observationoffset'], right_on = ['patientunitstayid','intakeoutputoffset'])
del merge, intakeOutputUrine
merge1

MemoryError: cannot allocate memory for array

In [ ]:
merge1['observationoffset'] = merge1.apply(lambda x: x['observationoffset'] if pd.notna(x['observationoffset']) else (x['labresultoffset'] if pd.notna(x['labresultoffset']) else x['intakeoutputoffset']), axis = 1)
merge1 = merge1.drop(['labresultoffset', 'intakeoutputoffset'], axis = 1)
merge1

In [ ]:
diagnosis = pd.read_csv('../eICU' + database_type + '/diagnosis.csv')
diagnosis = diagnosis.astype({'diagnosis': int})
diagnosis

In [ ]:
finalMerge = pd.merge(merge1, diagnosis,  how='outer', left_on=['patientunitstayid'], right_on = ['patientunitstayid'])
del merge1, diagnosis
finalMerge

In [ ]:
path = "../eICU/training/"
finalMerge.to_csv(path + "finalMerge.csv", sep=',', index=False, encoding='utf-8')
